In [1]:
%run utils.py
%matplotlib inline
import os.path as osp
import pandas as pd
import numpy as np
import tqdm
import mygene

In [2]:
mg = mygene.MyGeneInfo()

In [3]:
df_pr = pd.read_csv(osp.join(DATA_DIR, 'pubmed_abstract_proteins_resolved.csv'))
df_pr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61175 entries, 0 to 61174
Data columns (total 8 columns):
id            61175 non-null int64
start         61175 non-null int64
end           61175 non-null int64
value         61175 non-null object
class         61175 non-null object
value_norm    61175 non-null object
value_lbl     61175 non-null object
value_sym     17228 non-null object
dtypes: int64(3), object(5)
memory usage: 3.7+ MB


In [4]:
# Use normalized raw value instead of HGNC-resolved value_lbl
qfld = 'value_norm'

## Build Queries

In [5]:
from unidecode import unidecode

def normalize_to_query(pr):
    # Normalize greeks and super/subscripts (TNFα -> TNFa, CX₃CR1 -> CX3CR1)
    # Quotes and ticks will result bad http requests
    return unidecode(pr).replace("'", "").replace('"', '')

df_query = pd.DataFrame([
    {'query': normalize_to_query(pr), qfld: pr}
    for pr in df_pr[qfld].unique()
])
df_query.head()

,query,value_norm
0,FCGRIIB,FCGRIIB
1,IL2,IL2
2,FOXP3,FOXP3
3,CD4,CD4
4,TCR,TCR


In [6]:
# Show proteins that normalized to the same query terms (which isn't a problem)
cts = df_query.groupby('query').size()
df_query[df_query['query'].isin(cts[cts > 1].index.values)].sort_values('query')

,query,value_norm
484,CD3E,CD3Ε
1087,CD3E,CD3Ɛ
2584,CX3CL1,CX₃CL1
2125,CX3CL1,CX3CL1
2586,CX3CR1,CX₃CR1
342,CX3CR1,CX3CR1
3402,DP2,DP₂
2556,DP2,DP2
3845,FOXP3,FOXP³
2,FOXP3,FOXP3


In [7]:
batch_size = 250
q_pr = df_query['query'].unique()
q_pr_n = len(q_pr)
q_pr = np.array_split(q_pr, q_pr_n//batch_size)
print(
    'Number of queries total = {}\nNumber of query batches = {}\nBatch size counts:\n{}'
    .format(q_pr_n, len(q_pr), pd.Series([len(b) for b in q_pr]).value_counts())
)

Number of queries total = 5244
Number of query batches = 20
Batch size counts:
262    16
263     4
dtype: int64


## Run Queries

In [8]:
# See full list: http://docs.mygene.info/en/latest/doc/data.html#species
SPECIES = {9606: 'human', 10090: 'mouse'}

def query(terms):
    df = mg.querymany(
        terms,
        scopes=["symbol", "retired", "name", "alias"],
        fields='symbol,name,taxid,ensembl.gene,alias', 
        species='human,mouse', 
        ensemblonly=True,
        as_dataframe=True
    )
    if 'taxid' in df:
        df['species'] = df['taxid'].apply(lambda v: None if pd.isnull(v) else SPECIES.get(int(v), 'unknown'))
    return df

df_res = []
for batch in tqdm.tqdm(q_pr):
    df_res.append(query(list(batch)))
    
df_res = pd.concat(df_res)

  0%|          | 0/20 [00:00<?, ?it/s]

querying 1-263...done.


  5%|▌         | 1/20 [00:02<00:44,  2.36s/it]

Finished.
153 input query terms found dup hits:
	[('IL2', 4), ('FOXP3', 5), ('CD4', 10), ('TCR', 4), ('SATB1', 4), ('CD14', 3), ('HLA', 10), ('PD1', 
102 input query terms found no hit:
	['FCGRIIB', 'PROGRAMMEDDEATH1', 'TFH2', 'TFH17', 'TFH1', 'LGLS', 'MHCI', 'APRCA', 'IMMUNOGLOBULING2A
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-263...done.


 10%|█         | 2/20 [00:04<00:41,  2.33s/it]

...

Finished.
61 input query terms found dup hits:
	[('BMI1', 5), ('TGFb]', 10), ('VLA1', 2), ('RD6', 2), ('JANUS', 10), ('SPRED1', 2), ('RELAXIN', 10),
188 input query terms found no hit:
	['DCIL12P70', 'TYPEIANDIIIFN', 'TREGULATORYCELLTYPE', 'ANGIOGENICGROWTHFACTOR', 'HPV16E6', 'ESCC', '
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-262...done.


 95%|█████████▌| 19/20 [00:49<00:02,  2.85s/it]

Finished.
61 input query terms found dup hits:
	[('IFNb1', 2), ('MCP3', 2), ('MIP3b', 2), ('Ab', 10), ('Ab1', 4), ('PDE4B', 2), ('PDE4A', 2), ('H1R'
187 input query terms found no hit:
	['AIG', 'TXA23', 'TXA51', 'HTYR', 'MTYR', 'A2.1', 'NONSELF', 'CTLASSOCIATEDAG4', 'CD3SFV', 'ALLERGEN
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-262...done.


100%|██████████| 20/20 [00:51<00:00,  2.57s/it]

Finished.
45 input query terms found dup hits:
	[('CARDIOLIPIN', 2), ('ID', 8), ('ABPA', 2), ('JUN', 10), ('NKRP1A', 2), ('CAK', 4), ('PHOSPHODIESTE
194 input query terms found no hit:
	['MELANOMAASSOCIATEDPROTEIN', 'OVALBUMINPEPTIDE', 'BSAB', 'THROMBOPLASTIN', 'APTT', 'P38aMITOGENACTI
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


In [9]:
df_res.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10509 entries, FCGRIIB to CYA
Data columns (total 10 columns):
_id             7174 non-null object
_score          7174 non-null float64
alias           6028 non-null object
ensembl         259 non-null object
ensembl.gene    5762 non-null object
name            7174 non-null object
notfound        3335 non-null object
symbol          7174 non-null object
taxid           7174 non-null float64
species         7174 non-null object
dtypes: float64(2), object(8)
memory usage: 903.1+ KB


In [10]:
df_res.head()

,_id,_score,alias,ensembl,ensembl.gene,name,notfound,symbol,taxid,species
query,,,,,,,,,,
FCGRIIB,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,None
IL2,3558,91.829506,"[IL-2, TCGF, lymphokine]",NaN,ENSG00000109471,interleukin 2,NaN,IL2,9606.0,human
IL2,16183,75.850266,Il-2,NaN,ENSMUSG00000027720,interleukin 2,NaN,Il2,10090.0,mouse
IL2,3702,12.175253,"[EMT, LPFS1, LYK, PSCTK2]",NaN,ENSG00000113263,IL2 inducible T cell kinase,NaN,ITK,9606.0,human
IL2,16428,10.497415,"[Emt, Tcsk, Tsk]",NaN,ENSMUSG00000020395,IL2 inducible T cell kinase,NaN,Itk,10090.0,mouse


In [11]:
df_qlkp = df_query.set_index('query')
df_qlkp.index.is_unique

False

In [12]:
df_qlkp.head()

,value_norm
query,
FCGRIIB,FCGRIIB
IL2,IL2
FOXP3,FOXP3
CD4,CD4
TCR,TCR


In [27]:
# Merge results by query string back to query frame containing original protein symbol
df_res_pr = pd.merge(
    df_res.reset_index(), 
    df_qlkp.reset_index(),
    on='query',
    how='left'
)

# Reorganize some field names
df_res_pr = df_res_pr.rename(columns={
    '_id': 'id', 
    '_score': 'score', 
    'ensembl.gene': 'ensemblgene',
    'symbol': 'sym'
})

# Remove results with no match
df_res_pr = df_res_pr[df_res_pr['id'].notnull()]
df_res_pr = df_res_pr[['id', 'score', 'sym', 'name', 'ensemblgene', 'species', qfld]]

df_res_pr['ensemblgene'] = df_res_pr['ensemblgene'].str.strip()

# Ensure that remaining records all have a reference to pubmed protein name
assert df_res_pr[qfld].notnull().all()

df_res_pr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7202 entries, 1 to 10538
Data columns (total 7 columns):
id             7202 non-null object
score          7202 non-null float64
sym            7202 non-null object
name           7202 non-null object
ensemblgene    5788 non-null object
species        7202 non-null object
value_norm     7202 non-null object
dtypes: float64(1), object(6)
memory usage: 450.1+ KB


In [28]:
df_res_pr.head()

,id,score,sym,name,ensemblgene,species,value_norm
1,3558,91.829506,IL2,interleukin 2,ENSG00000109471,human,IL2
2,16183,75.850266,Il2,interleukin 2,ENSMUSG00000027720,mouse,IL2
3,3702,12.175253,ITK,IL2 inducible T cell kinase,ENSG00000113263,human,IL2
4,16428,10.497415,Itk,IL2 inducible T cell kinase,ENSMUSG00000020395,mouse,IL2
5,50943,89.723680,FOXP3,forkhead box P3,ENSG00000049768,human,FOXP3


In [29]:
# Validate that all species results were resolved
assert df_res_pr['species'].isin(SPECIES.values()).all()
df_res_pr['species'].value_counts()

human    4306
mouse    2896
Name: species, dtype: int64

## Add transcription factor flag

In [31]:
# This is small enough that it doesn't need a separate annotation notebook
df_tf = pd.read_csv(osp.join(DATA_DIR, 'human_tfs.csv'))
df_tf['ensemblid'] = df_tf['ensemblid'].str.strip()
df_res_pr['is_tf'] = df_res_pr['ensemblgene'].isin(df_tf['ensemblid'].unique())
df_res_pr['is_tf'].value_counts()

False    6849
True      353
Name: is_tf, dtype: int64

## Export

In [35]:
# For consistency with other annotation notebooks, prefix field originating
# from pubmed protein table
df_exp = df_res_pr.rename(columns={qfld: 'pr_' + qfld})
df_exp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7202 entries, 1 to 10538
Data columns (total 8 columns):
id               7202 non-null object
score            7202 non-null float64
sym              7202 non-null object
name             7202 non-null object
ensemblgene      5788 non-null object
species          7202 non-null object
pr_value_norm    7202 non-null object
is_tf            7202 non-null bool
dtypes: bool(1), float64(1), object(6)
memory usage: 457.2+ KB


In [33]:
path = osp.join(DATA_DIR, 'pubmed_mygene_protein_annotations.csv')
df_res_pr.to_csv(path, index=False)
path

'data/pubmed_mygene_protein_annotations.csv'